# Assignment 1 : Decision Tress using `scikit-learn`

Scikit-learn provides a range of supervised and unsupervised learning algorithms via a consistent interface in Python. In this assigment you'll explore how to train decision trees using the `scikit-learn` library. The scikit-learn documentation can be found [here](http://scikit-learn.org/stable/documentation.html).

In this assignment we'll attempt to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains 1151 instances and 20 attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

In [255]:
#You may add additional import if you want
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import time
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.model_selection import GridSearchCV
from sklearn import svm

In [256]:
%matplotlib inline

In [257]:
# Read the data from csv file
col_names = []
for i in range(20):
    if i == 0:
        col_names.append('quality')
    if i == 1:
        col_names.append('prescreen')
    if i >= 2 and i <= 7:
        col_names.append('ma' + str(i))
    if i >= 8 and i <= 15:
        col_names.append('exudate' + str(i))
    if i == 16:
        col_names.append('euDist')
    if i == 17:
        col_names.append('diameter')
    if i == 18:
        col_names.append('amfm_class')
    if i == 19:
        col_names.append('label')

data = pd.read_csv("messidor_features.txt", names = col_names)
print(data.shape)
data.head(10)

(1151, 20)


,quality,prescreen,ma2,ma3,ma4,ma5,ma6,ma7,exudate8,exudate9,exudate10,exudate11,exudate12,exudate13,exudate14,exudate15,euDist,diameter,amfm_class,label
0,1,1,22,22,22,19,18,14,49.895756,17.775994,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1,0
1,1,1,24,24,22,18,16,13,57.709936,23.799994,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0,0
2,1,1,62,60,59,54,47,33,55.831441,27.993933,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0,1
3,1,1,55,53,53,50,43,31,40.467228,18.445954,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0,0
4,1,1,44,44,44,41,39,27,18.026254,8.570709,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0,1
5,1,1,44,43,41,41,37,29,28.356400,6.935636,2.305771,0.323724,0.000000,0.000000,0.000000,0.000000,0.502831,0.126741,0,1
6,1,0,29,29,29,27,25,16,15.448398,9.113819,1.633493,0.000000,0.000000,0.000000,0.000000,0.000000,0.541743,0.139575,0,1
7,1,1,6,6,6,6,2,1,20.679649,9.497786,1.223660,0.150382,0.000000,0.000000,0.000000,0.000000,0.576318,0.071071,1,0
8,1,1,22,21,18,15,13,10,66.691933,23.545543,6.151117,0.496372,0.000000,0.000000,0.000000,0.000000,0.500073,0.116793,0,1
9,1,1,79,75,73,71,64,47,22.141784,10.054384,0.874633,0.099780,0.023386,0.000000,0.000000,0.000000,0.560959,0.109134,0,1


### 1. Data preprocessing  & dimensionality reduction with PCA

Q1. Separate the feature columns from the class label column. You should end up with two separate data frames (or two lists, or two numpy arrays) - one that contains all of the feature values and one that contains the class labels.

In [258]:
# creat the list data_header containing the name of each column
data_header = data.pop('label')

In [259]:
#create a numpy ndarray of data_values containing the values of each column
data_values = data.values
#data_values.shape
print(data_header.shape)
print(data_values.shape)

(1151,)
(1151, 19)


Q2. Use `sklearn.preprocessing.StandardScaler` to standardize the dataset’s features (mean = 0 and variance = 1). Only standardize the the features, not the class labels! This will be required for running the principal component analysis (PCA) dimensionality reduction later. Note that `StandardScaler` returns a numpy array.

In [260]:
# standardize the dataser's features. 
#http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
scaler = StandardScaler()
scaler.fit(data)
data_standardized = scaler.transform(data)
#print(np.average(data_standardized))
#print(data_standardized.var())
data_standardized.shape

(1151, 19)

Q3 . Split your dataset into training and test sets (80% - 20% split). Use `sklearn.model_selection.train_test_split` to help you in this task. You should be working with your standardized features from here forward. Display how many records are in the training set and how many are in the test set.

In [261]:
# Split standardized dataset into training (80%) and test (20%) sets. 
# http://scikit-learn.org/0.16/modules/generated/sklearn.cross_validation.train_test_split.html
data_train, data_test, labels_train, labels_test = train_test_split(data_standardized, data_header, test_size = 0.2)
print(data_train.shape)
print(data_test.shape)
#print("training set size: ",data_train.size)
#print("test set size: ",data_test.size)

(920, 19)
(231, 19)


In [262]:
print(data_train[:2])

[[ 0.05905386  0.2982129  -1.22720003 -1.24131448 -1.23448744 -1.19859857
  -1.16647699 -1.13621559 -0.47677523 -0.54135276 -0.64097312 -0.40540237
  -0.15987475 -0.18103623 -0.21496782 -0.2080998   0.22753328  0.62678629
  -0.7117194 ]
 [ 0.05905386  0.2982129   0.33470325  0.33576717  0.43250891  0.45972967
   0.37192536  0.32122077 -0.60883081 -0.71376463 -0.61681559 -0.44751361
  -0.22582799 -0.20090532 -0.21496782 -0.2080998  -0.38159912  1.50424372
  -0.7117194 ]]


Q4. PCA is affected by the scale of the features that is why it is important to standardize the dataset first. The principle components generated by PCA are sensitive to the shape of the data in d-dimensional space. 
* Carry out a principal components analysis using `sklearn.decomposition.PCA` Note that you are fitting PCA on the **training set** only (NOT the test set).
* Use the `pca.explained_variance_ratio_` field to determine how many principal components are needed so that 95% variance is retained. 
* Reduce the PCA-transformed-dataset to this number of columns and you'll use the resulting dataset for subsequent tasks.

* Once the training set's dimensionality has been reduced with PCA, transform the **test set** to the principal component space that was created. (Do not fit a new PCA. Use the same one that was created with the training set.)

In [263]:
# Principal component analysis (PCA) using sklearn.decomposition.PCA to the TRAINING SET.
#http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
pca = PCA()
data_train_pca = pca.fit(data_train)
#Determine how many principal components are needed so 95% variance is retained.
components = data_train_pca.explained_variance_ratio_
#print(components)
#print(components.size)
k = 0;
variance = 0;
for i in range(components.size):
    if variance < 0.95:
        variance = variance + components[i]
        k += 1;
print(variance)
print(k)
#Method #2 -> gives a variance larger than 95% and 10 components.
#pca2 = PCA(.95)
#data_train_reduced = pca2.fit(data_train)
#pca2.n_components_

0.9690769932249502
9


In [264]:
#Reduce PCA to components
#http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA.fit
pca_reduced = PCA(n_components=9)
pca_reduced.fit(data_train)
data_train_reduced = pca_reduced.transform(data_train)
#data_train_reduced[0]

In [265]:
#Transforming the test set using the same PCA used for the training set. 
#http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA.fit
data_test_reduced = pca_reduced.transform(data_test)
#data_test_reduced[0]

### 2. Training Decision Trees in `scikit-learn`

Q5. Use `sklearn.tree.DecisionTreeClassifier` to fit a decision tree classifier on the training set. Use entropy as the split criterion. 

In [266]:
#http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
#http://scikit-learn.org/stable/modules/tree.html#tree
clf = tree.DecisionTreeClassifier(criterion = 'entropy')
clf.fit(data_train_reduced, labels_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Q6. Now that the tree has been learned from the training data, we can run the test data through and predict classes for our test data. Use the `predict` method of `DecisionTreeClassifier` to classify the test data. Then use `sklearn.metrics.accuracy_score` to print out the accuracy of the classifier on the test set.

In [267]:
#predict classes for test data
#http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.predict
pred = clf.predict(data_test_reduced)

In [268]:
#print out accuracy of the classifier on the test set. 
#http://scikit-learn.org/stable/modules/model_evaluation.html#accuracy-score
#http://scikit-learn.org/stable/modules/model_evaluation.html#accuracy-score
accuracy = accuracy_score(labels_test, pred)
print(accuracy)

0.5974025974025974


Q7. Note that the DecisionTree classifier has many parameters that can be set. Try tweaking parameters like split criterion, max_depth, min_impurity_decrease, min_samples_leaf, min_samples_split, etc. to see how they affect accuracy.

In [269]:
#http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
clf2 = tree.DecisionTreeClassifier(criterion = 'entropy', splitter="random", max_depth=5, max_leaf_nodes=15)
clf2.fit(data_train_reduced, labels_train)
pred2 = clf2.predict(data_test_reduced)
accuracy2 = accuracy_score(labels_test, pred2)
print(accuracy2)

0.5887445887445888


### 3. Using K-fold Cross Validation

You have now built a decision tree and tested it's accuracy using the "holdout" method. But as discussed in class, this is not sufficient for estimating generalization accuracy. Instead, we should use Cross Validation to get a better estimate of accuracy. You will do that next.

Q7. Use `sklearn.model_selection.cross_val_score` to perform 10-fold cross validation on your decision tree. Report the accuracy of the model. For this step, revert back to using the data before it underwent PCA. So you want to use the standardized data that came out of Q2. 

In [270]:
#Use data_standardized data 
print(data_standardized.shape)
print(data_header.shape)
#cross validation measures accuracy, it does not improve it.
#http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
result = cross_validation.cross_val_score(clf, data_standardized, data_header, cv=10)

(1151, 19)
(1151,)


In [271]:
#getting the accuracy of the 10-fold cross validation
np.average(result)

0.6160119940029987

Q8. Now we want to tune our model to use the best parameters to avoid overfitting to our training data. Grid search is an approach to parameter tuning that will methodically build and evaluate a model for each combination of algorithm parameters specified in a grid. 
* Use `sklearn.model_selection.GridSearchCV` to find the best `max_depth`, `max_features`, and `min_samples_leaf` for your tree. Use 'accuracy' for the scoring criteria.
* Try the values [5,10,15,20] for `max_depth` and `min_samples_leaf`. Try [5,10,15] for `max_features`. 
* Use a 5-fold cross validation. 
* Print out the best value for each of the tested parameters.
* Print out the accuracy of the model with these best values.

In [274]:
# your code goes here
#https://chrisalbon.com/machine_learning/model_evaluation/cross_validation_parameter_tuning_grid_search/
#http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
params = { 'max_depth':[5,10,15,20],'max_features':[5,10,15], 'min_samples_leaf':[5,10,15,20]}
grid = GridSearchCV(clf, params, cv=5, scoring='accuracy')
grid.fit(data_standardized, data_header)

GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [5, 10, 15, 20], 'max_features': [5, 10, 15], 'min_samples_leaf': [5, 10, 15, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [273]:
#printing accuracy score and best values
print('Accuracy: ', grid.best_score_)
print('Best max_depth: ', grid.best_estimator_.max_depth)
print('Best max_features: ', grid.best_estimator_.max_features)
print('Best min_samples_leaf: ', grid.best_estimator_.min_samples_leaf)

Accuracy:  0.6507384882710686
Best max_depth:  20
Best max_features:  10
Best min_samples_leaf:  20


Q9. To perform the nested cross-validation that we discussed in class, you'll now need to pass the `GridSearchCV` into a `cross_val_score`. 

What this does is: the `cross_val_score` splits the data in to train and test sets for the first fold, and it passes the train set into `GridSearchCV`. `GridSearchCV` then splits that set into train and validation sets for k number of folds (the inner CV loop). The hyper-parameters for which the average score over all inner iterations is best, is reported as the `best_params_`, `best_score_`, and `best_estimator_`(best decision tree). This best decision tree is then evaluated with the test set from the `cross_val_score` (the outer CV loop). And this whole thing is repeated for the remaining k folds of the `cross_val_score` (the outer CV loop). 

That is a lot of explanation for a very complex (but IMPORTANT) process, which can all be performed with a single line of code!

Be patient for this one to run. The nested cross-validation loop can take some time. An asterisk [\*] next to the cell indicates that it is still running.

Print the accuracy of your tuned, cross-validated model. This is the official accuracy that you would report for your model.

In [275]:
#http://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html
grid_cross_val = cross_validation.cross_val_score(grid, data_standardized, data_header, cv=5)

In [276]:
#Accuracy of the tuned, cross-validated model
np.average(grid_cross_val)

0.6394202898550725

Q10. Our accuracy rate isn't very good. We wouldn't want to use this model in the real world to actually diagnose patients because it is wrong about 40% of the time! What can we do to improve the accuracy of the model? Answer as a comment:

In [ ]:
'''
Some of the things we could do to improve accuracy on the model are:
- Get more data so the model can learn better.
- Try a different algorithm
- Analyse the values that got wrong and try to find a pattern.
- re-engineer features. 
'''